<a href="https://colab.research.google.com/github/julianolm/Local-Binary-Convolutional-Neural-Network/blob/main/LBCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptacao e melhora de performance da primeira LBCNN treinavel 
(e aplicacao ao dataset cifar10)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [4]:
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Training on device {device}.')

Training on device cuda.


## Preparando o Dataset

In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
cifar10_val = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=32, shuffle=False)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Modelo

In [7]:
def new_kernel(in_channels=384, inter_channels=512, sparsity=0.1):
    """
        Retorna os anchor weights para uma camada de "in_channels" canais de entrada e um canal de saida.
    """
    from random import randint
    from math import ceil

    nk = np.zeros((inter_channels, in_channels*3*3))
    for out in range(inter_channels):
        for i in range(ceil(in_channels*3*3*sparsity)):
            a = randint(0,in_channels*3*3 - 1)
            nk[out, a] = 1 if randint(1,2)==1 else -1

    nk = nk.reshape(inter_channels, in_channels, 3, 3)
    return nk

In [8]:
class LBCBlock(nn.Module):
    def __init__(self, n_channels=384, n_kernels=512, sparsity=0.1):
        super().__init__()
        self.n_channels = n_channels
        self.n_kernels = n_kernels
        self.Batch_Norm = nn.BatchNorm2d(n_channels)

        self.conv_filter = nn.Conv2d(n_channels, n_kernels, kernel_size=3, padding=1, bias=False)
        kernels = torch.tensor(new_kernel(n_channels, n_kernels, sparsity)).type('torch.FloatTensor')
        self.conv_filter.weight = nn.Parameter(kernels, requires_grad=False)

        self.weighted_sum = nn.Conv2d(n_kernels, n_channels, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.Batch_Norm(x)
        with torch.no_grad():
            out = torch.relu(self.conv_filter(out))
        out = self.weighted_sum(out)
        out += x

        return out

In [9]:
img_width = 32
img_height = 32

class NetLBC(nn.Module):
    def __init__(self, lbc_filters=512, n_channels=384, n_blocks=10, sparsity=0.1, hidden_units=384):
        super().__init__()
        self.n_channels = n_channels
        self.n_blocks = n_blocks
        self.hidden_units = hidden_units

        self.conv1 = nn.Conv2d(3, n_channels, kernel_size=3, padding=1)
        # self.lbc_blocks = nn.Sequential(*(self.n_blocks * [LBCBlock(n_channels, lbc_filters, sparsity)]))
        # self.lbc_blocks = nn.Sequential(*( [LBCBlock(n_channels, lbc_filters, sparsity) for i in range(n_blocks)]))
        from collections import OrderedDict
        self.lbc_blocks = nn.Sequential(
            OrderedDict([
                         (f'lbc_block_{i}', LBCBlock(n_channels, lbc_filters, sparsity)) for i in range(1,n_blocks+1) 
                         ])
            )

        self.fc1 = nn.Linear(self.n_channels * 6 * 6, self.hidden_units)                                      #---------opcao 2
        self.fc2 = nn.Linear(self.hidden_units, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.lbc_blocks(out)
        out = torch.nn.functional.max_pool2d(out, kernel_size=6,padding=2)                      #---------opcao 2
        
        out = out.view(-1, self.n_channels * 6 * 6)                                             #---------opcao 2

        out = torch.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [10]:
from datetime import datetime as datetime

def training_loop(n_epochs, model, loss_fn, train_loader, optimizer, scheduler=None):
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        iter = 0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            out = model(imgs)
            loss = loss_fn(out, labels)

            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()

            # torch.nn.utils.clip_grad_value_(model.parameters(), 10)
            optimizer.step()

            # print(f"Epoch: {epoch} || Batch number: {iter} || Batch Loss: {loss.item()}")
            
            iter += 1

        if scheduler is not None:
            scheduler.step()

        print("%s Epoch: %d, Loss: %f" % (datetime.now(), epoch, train_loss/len(train_loader)))

In [11]:
def validate(model, train_loader, val_loader):
    for season, loader in [('Train', train_loader), ('Validation', val_loader)]:
        total = 0
        correct = 0
        for imgs, labels in loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            with torch.no_grad():
                out = model(imgs)
                _, pred = torch.max(out, dim=1)

                total += imgs.shape[0]
                correct += int((pred == labels).sum())

        print("%s accuracy: %f" % (season, correct/total*100))

In [ ]:
#  torch.autograd.set_detect_anomaly(True)

In [28]:
model = NetLBC(lbc_filters=512, n_channels=16, n_blocks=10, sparsity=0.1, hidden_units=384).to(device=device)

In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

In [30]:
model.load_state_dict(torch.load('/content/drive/MyDrive/LBCNN_1.1.pt'))

<All keys matched successfully>

In [31]:
validate(model, train_loader, val_loader)

Train accuracy: 91.152000
Validation accuracy: 79.230000


##Treinamento

In [ ]:
loss = nn.CrossEntropyLoss()
learnable_parameters = []
for _, param in model.named_parameters():
    learnable_parameters.append(param)
optimizer = torch.optim.SGD(params=learnable_parameters, lr=1e-4, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
training_loop(n_epochs=180,
              model=model,
              loss_fn=loss,
              train_loader=train_loader,
              optimizer=optimizer,
              scheduler=scheduler)

2021-03-03 13:55:27.503103 Epoch: 1, Loss: 9.425127
2021-03-03 14:19:25.729173 Epoch: 2, Loss: 2.302821
2021-03-03 14:43:24.159777 Epoch: 3, Loss: 2.302785
2021-03-03 15:07:22.274935 Epoch: 4, Loss: 2.302765
2021-03-03 15:31:19.658889 Epoch: 5, Loss: 2.302730
2021-03-03 15:55:17.113904 Epoch: 6, Loss: 2.302718


KeyboardInterrupt: ignored

In [ ]:
model_save_name = 'LBCNN_384chan_50blck_01spar_512hidd.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

FileNotFoundError: ignored

In [ ]:
validate(model, train_loader, val_loader)

Train accuracy: 10.000000
Validation accuracy: 10.000000


## Salvando modelo

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# model_save_name = 'LBCNN_1.2.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)